In [ ]:
import sys, copy
import numpy as np
import torch
import random
from torch.autograd import Variable

class One_Chain_Optimizer:
    def __init__(self, net, criterion, momentum=0.5, lr=1e-4, wdecay=5e-4, T=0.5, total=1000):
        self.net = net
        self.eta = lr
        self.momentum = momentum
        self.T = T
        self.wdecay = wdecay
        self.V = 0.1
        self.velocity = []
        self.criterion = criterion
        self.total = total

        self.beta = 0.5 * self.V * self.eta
        self.alpha = 1 - self.momentum

        if self.beta > self.alpha:
            sys.exit('Momentum is too large')

        self.sigma = np.sqrt(2.0 * self.eta * (self.alpha - self.beta))
        self.scale = self.sigma * np.sqrt(self.T)

        for param in net.parameters():
            p = torch.zeros_like(param.data)
            self.velocity.append(p)

    def set_T(self, factor=1):
        self.T /= factor
        self.scale = self.sigma * np.sqrt(self.T)

    def set_eta(self, eta):
        self.eta = eta
        self.beta = 0.5 * self.V * self.eta
        self.sigma = np.sqrt(2.0 * self.eta * (self.alpha - self.beta))
        self.scale = self.sigma * np.sqrt(self.T)

    def backprop(self, x, y):
        self.net.zero_grad()
        """ convert mean loss to sum losses """
        loss = self.criterion(self.net(x), y)  # * self.total
        loss.backward()
        return loss

    def step(self, x, y):
        loss = self.backprop(x, y)
        print("Check loss:", loss)
        for i, param in enumerate(self.net.parameters()):
            proposal = torch.FloatTensor(param.data.size()).normal_().mul(self.scale)
            grads = param.grad.data
            if self.wdecay != 0:
                grads.add_(self.wdecay, param.data)
            self.velocity[i].mul_(self.momentum).add_(-self.eta, grads).add_(proposal)
            param.data.add_(self.velocity[i])
        return loss.data.item()

In [ ]:
import math
import copy
import sys
import os
import time
import csv
import argparse
import random
import collections
from random import shuffle
import pickle

from tqdm import tqdm ## better progressbar
from math import exp
from sys import getsizeof
import numpy as np

## import pytorch modules
import torch
from torch.autograd import Variable
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.utils.data as data
import torchvision.datasets as datasets

CUDA_EXISTS = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Your own Parameters in parameters : parameters = [T, lr, num_of_chains, wdecay, total, Tgap, LRgap, num_epoch, period, batch, var_reduce, adapt_c, alpha, bias_F, cool, burn, Tanneal, LRanneal]
# T:Temperature for high temperature chain, default=0.05
# lr: Sampling learning rate, default=0.1
# num_of_chains: Total number of chains, default=1
# wdecay: Samling weight decay, default=5e-4
# total: Total data points, default=50000
# Tgap: Temperature gap between chains, default=0.2
# LRgap: Learning rate gap between chains, default=0.66
# num_epoch: Sampling Epochs, default=1000
# period: estimate adaptive variance every [period] epochs, default=2
# batch: Batch size, default=256
# var_reduce: n>0 means update variance reduction every n epochs; n divides 10, default=0
# adapt_c: adapt_c=1 is equivalent to running Alg. 2 in the appendix, default=0
# alpha: forgetting rate, default=0.3
# bias_F: correction factor F, default=1.5e5
# cool: No swaps happen during the cooling time after a swap, default=1
# burn: burn in iterations for sampling (sn * burn), default=0.6
# Tanneal: temperature annealing factor, default=1.02
# LRanneal: lr annealing factor, default=0.984

def trainer(nets, training_data, parameters):
  # Loss function
  criterion = nn.MSELoss()
  # Initial temperature and learning rate
  init_T, init_lr = parameters.T, parameters.lr

  chains, lr_set, myVars, cooling_time, BMAS = {}, [], [], [], []
  for idx in range(parameters.num_of_chains-1, -1, -1):
    print('Chain {} Initial learning rate {:.2e} temperature {:.2e}'.format(idx, init_lr, init_T))
    chain = One_Chain_Optimizer(nets[idx], criterion)
    lr_set.insert(0, init_lr)
    init_T /= parameters.Tgap
    init_lr /= parameters.LRgap
    chains[idx] = chain
    myVars.append(sys.float_info.max)

  start = time.time()
  counter, warm_up, adjusted_corrections = 1., 10, 0
  # Initialization for variance reduction
  last_full_losses, last_VRnets, corr = [0] * parameters.num_of_chains, [], [-1] * parameters.num_of_chains
  for idx in range(parameters.num_of_chains):
    last_VRnets.append(pickle.loads(pickle.dumps(nets[idx])))

  for epoch in range(parameters.num_epoch):
    # Update adaptive variance and variance reduction every [period] epochs
    if parameters.period > 0 and epoch % parameters.period ==0:
      cur_full_losses = [0] * parameters.num_of_chains
      for idx in range(parameters.num_of_chains):
        stage_losses, cv_losses = [], []
        nets[idx].eval()
        for i, (x, y) in enumerate(training_data):
          x = x.to(device)
          y = y.to(device)
          nets[idx].zero_grad()
          avg_loss = criterion(nets[idx](x), y).item()
          cur_full_losses[idx] += avg_loss * parameters.batch
          stage_losses.append(avg_loss * parameters.total)


          if parameters.var_reduce:
            cv_losses.append(criterion(last_VRnets[idx](x), y).item() * parameters.total)

          if parameters.adapt_c:
            adaptive_corr = -np.cov(stage_losses, cv_losses, ddof=1)[0][1] / np.var(cv_losses, ddof=1)
            corr[idx] = (1 - parameters.alpha) * corr[idx] + parameters.alpha * adaptive_corr

          if parameters.var_reduce:
            for i in range(len(stage_losses)):
              stage_losses[i] = stage_losses[i] + corr[idx] * (cv_losses[i] - np.mean(cv_losses))

        std_epoch = np.std(stage_losses, ddof=1)

        myVars[idx] = 0.5 * std_epoch**2 if myVars[idx] == sys.float_info.max else ((1 - parameters.alpha) * myVars[idx] + parameters.alpha * 0.5 * std_epoch ** 2)

        print('Epoch {} Chain {} loss std {:.2e} variance {:.2e} smooth variance {:.2e} adaptive c {:.2f}'.format(epoch, idx, std_epoch, 0.5 * std_epoch**2, myVars[idx], corr[idx]))
        last_VRnets[idx] = pickle.loads(pickle.dumps(nets[idx]))
        last_full_losses[idx] = cur_full_losses[idx]

    for idx in range(parameters.num_of_chains):
      nets[idx].train()

    for i, (x, y) in enumerate(training_data):
      x = x.to(device)
      y = y.to(device)
      counter += 1
      loss_chains = []
      for idx in range(parameters.num_of_chains):
        loss = chains[idx].step(x, y)
        # variance-reduced negative log posterior
        if parameters.var_reduce and epoch > warm_up:
          control_variate_loss = criterion(last_VRnets[idx](x), y).item() * parameters.total
          loss = loss + corr[idx] * (control_variate_loss - last_full_losses[idx])
        loss_chains.append(loss)

      # Swap

      for idx in range(parameters.num_of_chains - 1):

        # exponential average smoothing
        delta_invT = 1. / chains[idx].T - 1. / chains[idx+1].T
        adjusted_corrections = delta_invT * (myVars[idx] + myVars[idx+1]) / parameters.bias_F
        print("Check first term:",  np.log(np.random.uniform(0, 1)))
        print("Check second term:", delta_invT * (loss_chains[idx] - loss_chains[idx+1] - adjusted_corrections))
        if np.log(np.random.uniform(0, 1)) < delta_invT * (loss_chains[idx] - loss_chains[idx+1] - adjusted_corrections):
          print("!!!!!!!!!!!!!!!!!!!!!!!!!!!")

          if epoch not in cooling_time:
            temporary = pickle.loads(pickle.dumps(chains[idx+1].net))
            chains[idx+1].net.load_state_dict(chains[idx].net.state_dict())
            chains[idx].net.load_state_dict(temporary.state_dict())
            print('Epoch {} Swap chain {} with chain {} and increased F {:0.2e}'.format(epoch, idx, idx+1, parameters.bias_F))
            cooling_time = range(epoch, epoch+parameters.cool)
          else:
            print('Epoch {} Cooling period'.format(epoch))

    # Anneaing]
    if epoch < parameters.burn * parameters.num_epoch:
      parameters.bias_F *= parameters.Tanneal
    for idx in range(parameters.num_of_chains):
      if epoch > 0.4 * parameters.num_epoch and parameters.LRanneal <=1:
        chains[idx].eta *= parameters.LRanneal
      if epoch < parameters.burn * parameters.num_epoch:
        chains[idx].set_T(parameters.Tanneal)

      # add test set here
      ##########################################

  end = time.time()
  print('Time used {:.2f}s'.format(end - start))

In [ ]:
parameters = argparse.Namespace(
    T=0.05,              # Lower initial temperature
    lr=1e-4,            # Lower learning rate
    num_of_chains=2,      # Only 1 chain for now
    wdecay=1e-4,          # Smaller weight decay to prevent over-penalizing weights
    total=50000,   # Total number of data points
    Tgap=0.2,             # Default Tgap
    LRgap=0.66,           # Default LRgap
    num_epoch=1000,        # Training epochs
    period=2,             # Period for variance reduction
    batch=256,             # Batch size
    var_reduce=0,         # No variance reduction
    adapt_c=0,            # No adaptive correction
    alpha=0.3,            # Forgetting rate
    bias_F=1.5e5,         # Correction factor
    cool=1,              # Cooling time for swaps
    burn=0.6,             # Burn-in period for annealing
    Tanneal=1.02,         # Temperature annealing factor
    LRanneal=0.984,         # Learning rate annealing factor
    momentum = 0.9
)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Use GPU if available, otherwise fall back to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create a simple linear dataset
np.random.seed(42)
x = np.random.rand(100, 1).astype(np.float32)
y = 3 * x + 1 + np.random.randn(100, 1).astype(np.float32) * 0.1  # Linear relationship y = 3x + 1 + noise

# Convert the numpy arrays to torch tensors
x_train = torch.tensor(x, dtype=torch.float32).to(device)
y_train = torch.tensor(y, dtype=torch.float32).to(device)

# Define a simple neural network model for regression
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(1, 50)  # 1 input feature to 10 hidden units
        self.fc2 = nn.Linear(50, 1)  # 10 hidden units to 1 output feature
        self.relu = nn.ReLU()  # ReLU activation

    def forward(self, x):
        x = self.relu(self.fc1(x))  # First layer + ReLU activation
        x = self.fc2(x)  # Second layer (output)
        return x

# Initialize the neural network
model = SimpleNN().to(device)

# Duplicate the model structure
model_duplicate = SimpleNN().to(device)

# Copy the weights from the original model to the duplicate
model_duplicate.load_state_dict(model.state_dict())

# Create a DataLoader to handle batching
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
training_data = DataLoader(train_dataset, batch_size=parameters.batch, shuffle=True)

criterion = nn.MSELoss()



Using device: cpu


In [ ]:
nets = []
nets.append(model)
nets.append(model_duplicate)

In [ ]:
trainer(nets, training_data, parameters)

Chain 1 Initial learning rate 1.00e-04 temperature 5.00e-02
Chain 0 Initial learning rate 1.52e-04 temperature 2.50e-01
Epoch 0 Chain 0 loss std nan variance nan smooth variance nan adaptive c -1.00
Epoch 0 Chain 1 loss std nan variance nan smooth variance nan adaptive c -1.00
Check loss: tensor(7.8027, grad_fn=<MseLossBackward0>)
Check loss: tensor(7.8027, grad_fn=<MseLossBackward0>)
Check first term: -0.2296505465363947
Check second term: nan
Check loss: tensor(7.7017, grad_fn=<MseLossBackward0>)
Check loss: tensor(7.5141, grad_fn=<MseLossBackward0>)
Check first term: -0.5500796041202634
Check second term: nan
Epoch 2 Chain 0 loss std nan variance nan smooth variance nan adaptive c -1.00
Epoch 2 Chain 1 loss std nan variance nan smooth variance nan adaptive c -1.00
Check loss: tensor(7.7592, grad_fn=<MseLossBackward0>)
Check loss: tensor(7.2620, grad_fn=<MseLossBackward0>)
Check first term: -1.633510404873745
Check second term: nan
Check loss: tensor(7.6299, grad_fn=<MseLossBackward0

KeyboardInterrupt: 

In [ ]:
delta_invT = 1 / 0.05 - 1 / 0.05

In [ ]:
stage_losses = [390.68483747541904]
std_epoch = np.std(stage_losses, ddof=1)
print("std_epoch",std_epoch)